# Jupyter Notebook - Electronics Demos
This notebook demonstrates several packages that can be used to support the creation of assets relating to *electronics*.

In [14]:
%matplotlib inline

## Drawing Electrical Circuit Diagrams / Schematics

One way of using the notebooks is an an envionment to support the creation of schematic diagrams.

### `schemdraw`

The `schemdraw` package can be used to write descriptions of electrical circuit diagrams.

Examples: https://cdelker.bitbucket.io/SchemDraw/SchemDraw.html


We can parameterise values to allow us to refer to diagrams in text using similar values.

In [15]:
import SchemDraw as schem
import SchemDraw.elements as e

In [16]:
#Would be nice if we could find a way of handling units better?
#Maybe a package that gets this from value and chooses unit modifier appropriately?

R=100 * 10**3
V=10
C=0.1 * 10**-6

V_str='{V}V'.format(V=V)
R_str='{R}K$\Omega$'.format(R=R/10**3)
C_str='{C}$\mu$F'.format(C=C/10**-6)
    
d = schem.Drawing()
V1 = d.add(e.SOURCE_V, label=V_str)
d.add(e.RES, d='right', label=R_str)
d.add(e.CAP, d='down', botlabel=C_str)
d.add(e.LINE, to=V1.start);

The following circuit shows a {{print(V_str)}} source connected in series to a {{R_str}} resistor and a {{print(C_str)}} capacitor.

{{d.draw()}}

*Double click on the cell to see how the components are embedded.*

We can also update diagrams dynamically:

In [17]:
from ipywidgets import interact
import ipywidgets
import matplotlib.pyplot as plt    

@interact(V=ipywidgets.IntSlider(min=0,max=15,step=1,value=5),
          R=ipywidgets.IntSlider(min=100,max=1500,step=100,value=100),
          C=ipywidgets.IntSlider(min=0,max=15,step=1,value=1))
def schem_demo(V, R, C, xkcdmode=False):
    if xkcdmode: plt.xkcd()
    d = schem.Drawing()
    V1 = d.add(e.SOURCE_V, label='{}V'.format(V))
    d.add(e.LINE, d='right', l=d.unit*.75)
    S1 = d.add(e.SWITCH_SPDT2_CLOSE, d='up', anchor='b', rgtlabel='$t=0$')
    d.add(e.LINE, d='right', xy=S1.c,  l=d.unit*.75)
    d.add(e.RES, d='down', label='${}\Omega$'.format(R), botlabel=['+','$v_o$','-'])
    d.add(e.LINE, to=V1.start)
    d.add(e.CAP, xy=S1.a, d='down', toy=V1.start, label='{}$\mu$F'.format(C))
    d.add(e.DOT)
    d.draw(showplot=False)

A Jupyter Widget

## Simulating Circuits

As well as drawing circuits, there several different approaches to simulating circuit behaviour.

## *Circuit Sandbox*

The [Circuit Sandbox](https://github.com/willymcallister/circuit-sandbox) is a simple, standalone Javascript circuit simulator that we can embed in a notebook or [run in its own browser tab](js/circuit-sandbox-master/index.html).

The *Circuit Sandbox* can save and load simple netlists (*I don't know if these can then be used by other packages?*)

In [18]:
from IPython.display import IFrame
IFrame('js/circuit-sandbox-master/index-mobile.html', width=1000,height=600)

## Digital Circuit Simulator

`simcirjs` [[docs](https://github.com/kazuhikoarase/simcirjs)) is a syandalone Javascript digital circuit simulator that we can embed in a notebook or [run in its own browser tab](js/simcirjs-master/sample.html).

In [19]:
#!ls js/simcirjs-master/
from IPython.display import IFrame
IFrame('js/simcirjs-master/blank.html', width=600,height=300)

### `ahkab`

[`ahkab`](https://ahkab.readthedocs.io/en/latest/index.html) is an open-source SPICE-like interactive circuit simulator.

*Unfortunately, it doesn't render the schematic diagram from the circuit description. It would be useful if we `ahkab` could work with something like `schemdraw` to support such a dual behaviour from the same circuit descrpition.*

The examples below are based on the original documentation.

In [20]:
import ahkab

In [21]:
cct = ahkab.Circuit('Simple Example Circuit')

cct.add_resistor('R1', 'n1', cct.gnd, value=5)
cct.add_vsource('V1', 'n2', 'n1', dc_value=8)
cct.add_resistor('R2', 'n2', cct.gnd, value=2)
cct.add_vsource('V2', 'n3', 'n2', dc_value=4)
cct.add_resistor('R3', 'n3', cct.gnd, value=4)
cct.add_resistor('R4', 'n3', 'n4', value=1)
cct.add_vsource('V3', 'n4', cct.gnd, dc_value=10)
cct.add_resistor('R5', 'n2', 'n4', value=4)

opa = ahkab.new_op()
r = ahkab.run(cct, opa)['op']
print(r)

OP simulation results for 'Simple Example Circuit'.
Run on 2017-12-04 15:44:15, data file /tmp/tmp5jo8a9gt.op.
Variable    Units        Value         Error    %
----------  -------  ---------  ------------  ---
VN1         V        -3.86364    3.86358e-12    0
VN2         V         4.13636   -4.13591e-12    0
VN3         V         8.13636   -8.13602e-12    0
VN4         V        10         -1.00027e-11    0
I(V1)       A        -0.772727   0              0
I(V2)       A        -0.170455   0              0
I(V3)       A        -3.32955    0              0


## Boolean Algebra - `PyEDA`

`PyEDA` is a [Python package for exploring Boolean algebra](http://pyeda.readthedocs.io/en/latest/boolalg.html).

In [22]:
from pyeda.inter import *

A = exprvar('A')
B = exprvar('B')
C = exprvar('C')

In [23]:
A & B

And(A, B)

In [24]:
A & ~B

And(A, ~B)

In [25]:
(A | B) & C

And(Or(A, B), C)

In [26]:
A ^ B

Xor(A, B)

In [27]:
expr2truthtable(A & ~B)

B A
0 0 : 0
0 1 : 1
1 0 : 0
1 1 : 0

In [28]:
expr2truthtable( Or(And(A, B), And(A, C)) )

C B A
0 0 0 : 0
0 0 1 : 0
0 1 0 : 0
0 1 1 : 1
1 0 0 : 0
1 0 1 : 1
1 1 0 : 0
1 1 1 : 1

In [29]:
expr2truthtable( And('X', 'Y') )

Y X
0 0 : 0
0 1 : 0
1 0 : 0
1 1 : 1

In [30]:
tt = truthtable(reversed([A,B,C]), "00010111")
tt

A B C
0 0 0 : 0
0 0 1 : 0
0 1 0 : 0
0 1 1 : 1
1 0 0 : 0
1 0 1 : 1
1 1 0 : 1
1 1 1 : 1

In [31]:
truthtable2expr( tt ) 

Or(And(~A, B, C), And(A, ~B, C), And(A, B, ~C), And(A, B, C))